In [1]:
machine = 'mac'
machine = 'magny'

In [2]:
if machine == 'magny':
    filedir = "/home/extmilan/masterthesis/files/"
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
    plotdir = "/home/extmilan/masterthesis/plots/"
elif machine == 'mac':
    filedir = "/Users/smilanov/Documents/masterthesis/auriga_files/files/"
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
else:
    raise NotADirectoryError

In [3]:
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, HernquistPotential, MWPotential2014
from galpy.actionAngle import estimateDeltaStaeckel, actionAngleStaeckel
from galpy.actionAngle import UnboundError

from areposnap.gadget import gadget_readsnap
from areposnap.gadget_subfind import load_subfind

from auriga_basics import *
from auriga_functions import *

import numpy as np
from IPython.display import Image, display

import corner

import datetime
import random

import copy

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.patches import Circle
from matplotlib import animation

import datetime

cmap = copy.copy(plt.cm.inferno)
cmap.set_bad((0,0,0))

%matplotlib inline

In [4]:
#### path = /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/*
level = 4
if machine == 'magny':
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
elif machine == 'mac': 
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
j = 0
for halo_number in [24]:  # range(1, 31):
    halodir = basedir+"halo_{0}/".format(halo_number)
    snappath = halodir+"output/"

    for snapnr in range(127,128,1):
        print("level   : {0}".format(level))
        print("halo    : {0}".format(halo_number))
        print("snapnr  : {0}".format(snapnr))
        print("basedir : {0}".format(basedir))
        print("halodir : {0}".format(halodir))
        print("snappath: {0}\n".format(snappath))
        s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
            haloid=0, galradfac=0.1, verbose=True) 

        # Clean negative and zero values of gmet to avoid RuntimeErrors
        # later on (e.g. dividing by zero)
        s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )
        

level   : 4
halo    : 24
snapnr  : 127
basedir : /hits/universe/GigaGalaxy/level4_MHD/
halodir : /hits/universe/GigaGalaxy/level4_MHD/halo_24/
snappath: /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/

[ 34.42281723  33.16259384  37.29567337]
Found 1783163 stars.
Rotated pos.
Rotated vel.

galrad  : 0.02408556640148163
redshift: 2.220446049250313e-16
time    : 0.9999999999999998
center  : [ 0.  0.  0.]



In [5]:
#_____reference values for galpy_____
_REFR0_kpc = 8.
_REFV0_kms = 220.


Action evolution parameters
---

- IDlist (somehow.... find where dwarf (67 für den Anfangs) merges)
- startnr
- fit routine = emcxee 1 (in v0.1; more maybe in v0.2)
- delta fix or calculation
- figure (with labels, range, title etc predefined)
- in v0.2 possibility for wrong potential


Get IDs of merged stars from snapshot 72 to 73
---
Maybe instead take table made in 2dwarf

In [6]:
ids_merger = np.loadtxt(filedir + 'surviving_ids_snapshot_73_sh_1.txt')

In [ ]:
'''
part_num = 100
rand_samp = np.array(random.sample(list(enumerate(ids_merger)), part_num))
rand_ind = rand_samp[:,0].astype(int)
IDS = rand_samp[:, 1]
'''
IDS = ids_merger
mergersnap = str(104)

In [ ]:

true_fit = True

snapnr = np.arange(74, 128, 1, dtype = int)



# startnr is number of first snapshot to calculate its actions (of the GCs with merger IDs)
startnr = 74 # worked for the first time in 93
# endnumber is last snapshot plus 1 (since I use range function)
endnr = int(np.max(snapnr) + 1) 
# NRSNAPSHOTS USELESS ATM
nrsnapshots = int(endnr-startnr)
# helping parameter to distinguish between first corner plot as base figure to plot other corner plots on
init = 0

# figure related color 
color = 'pink'


halo_number = 24  # range(1, 31):
halodir = basedir + "halo_{0}/".format(halo_number)
snappath = halodir + "output/"
for snap_number in range(startnr, endnr, 1):
    title = 'actions_merger_' + mergersnap + '_' + str(len(IDS)) + '_parts_snap_' + str(snap_number) + '_MWpot.txt'    
    print("snapnr: {0}".format(snap_number))
    print(datetime.datetime.now().time())
    s1, sf1 = eat_snap_and_fof(level, halo_number, snap_number, snappath, loadonlytype=[4], 
        haloid=0, galradfac=0.1, verbose=False) 

    # Clean negative and zero values of gmet to avoid RuntimeErrors
    # later on (e.g. dividing by zero)
    s1.data['gmet'] = np.maximum( s1.data['gmet'], 1e-40 ) 

    # create masks for merged stars according to their IDs to get their positions
    gcmask = np.isin(s1.id, IDS)
    print(np.sum(gcmask))
    outids = s1.id[gcmask]
    
    # set up galpy MWpotential
    pot_galpy = MWPotential2014
    
    # this would mean that there are no merged GCs
    if np.sum(gcmask) == 0:
        continue
        
    # get position and velocities of all selected GCs & convert to galpy units
    (R_kpc, phi_rad, z_kpc), (vR_kms, vphi_kms, vz_kms) = get_cylindrical_vectors(s1, sf1, gcmask)
    # convert physical to galpy units by dividing by REF vals (get velocities from best fit parameters)
    R_galpy, vR_galpy, vT_galpy, z_galpy, vz_galpy = R_kpc / _REFR0_kpc, vR_kms / _REFV0_kms, vphi_kms / _REFV0_kms, z_kpc / _REFR0_kpc, vz_kms / _REFV0_kms

    # estimate Delta of the Staeckel potential
    delta = 0.45
    delta = estimateDeltaStaeckel(pot_galpy, R_galpy, z_galpy)
    # CHECK HOW BIG INFLUENCE OF DELTA IS
    
    
    # set up the actionAngleStaeckel object
    aAS = actionAngleStaeckel(
            pot   = pot_galpy,  # potential
            delta = delta,      # focal length of confocal coordinate system
            c     = True        # use C code (for speed)
            )
    #### calculate actions (in galpy units) & convert them to physical units
    # CONTINUE COMMENTING FROM HERE
    r_condition = np.where((np.sqrt(R_kpc**2 + z_kpc**2)) <= (2000. * s1.galrad))
    jR_galpy, lz_galpy, jz_galpy, r_kpc = np.zeros(len(IDS)), np.zeros(len(IDS)), np.zeros(len(IDS)), np.zeros(len(IDS))
    savedids = np.zeros(len(IDS))
    ids_notworking = []
    for test_i, item in enumerate(IDS):
        if (test_i % 1000) == 0:
            print(datetime.datetime.now().time())
            print(test_i)
        if np.sum(np.isin(r_condition, test_i)):
            try: 
                jR_galpy[test_i], lz_galpy[test_i], jz_galpy[test_i] = aAS(R_galpy[test_i], vR_galpy[test_i], vT_galpy[test_i], z_galpy[test_i], vz_galpy[test_i])
                r_kpc[test_i] = np.sqrt(R_kpc[test_i]**2 + z_kpc[test_i]**2)
                savedids[test_i] = item
            except(ValueError, UnboundError):
                #print('did not work!')
                ids_notworking.append(item)
                continue
    print('numbers of GCs wo actions:', len(ids_notworking))
    jR_kpckms, lz_kpckms, jz_kpckms = jR_galpy * _REFR0_kpc * _REFV0_kms, lz_galpy * _REFR0_kpc * _REFV0_kms, jz_galpy * _REFR0_kpc * _REFV0_kms
    
    
    #savedids[test_i] = item
            #except ValueError:
                #continue
    survivor_id_mask = np.isin(IDS, savedids)
    jR_kpckms, lz_kpckms, jz_kpckms = jR_kpckms[survivor_id_mask], lz_kpckms[survivor_id_mask], jz_kpckms[survivor_id_mask]
    r_kpc = r_kpc[survivor_id_mask]
    survivor_ids = IDS[survivor_id_mask]
    print(len(survivor_ids))
    #print(2000. * s1.galrad)
    # prepare data for plotting and create corner plot
    '''    
    data = np.vstack([jR_kpckms, lz_kpckms, jz_kpckms])
    
    print(data)
    labels = ['jR [kpc km/s]', 'lz [kpc km/s]', 'jz [kpc km/s]']

    figure = corner.corner(data.transpose(), labels = labels, plot_contours = 0, norm=matplotlib.colors.LogNorm(), cmap = cmap)
    display(figure)
    '''
    if len(survivor_ids) != 0:
        np.savetxt(filedir + title, (survivor_ids, r_kpc, jR_kpckms, lz_kpckms, jz_kpckms), fmt='%.18f', header='ID\tr [kpc]\tjR [kpc km/s]\tlz [kpc km/s]\tjz [kpc km/s]')

    '''
    to read in these files use: 
    (but in a loop replace 93 by snapnr then you don't have to extract it from the string adn can loop over all the files)
    infile = '/home/extmilan/masterthesis/files/actions_snap_93_nonsmoothed_pot_params.txt'
    ids, r, jr, jphi, jz = np.loadtxt('/home/extmilan/masterthesis/files/actions_snap_93_nonsmoothed_pot_params.txt')
    snapshot = int(infile[(infile.find('snap_') + 5):].split('_')[0])
    ids = np.array(ids, dtype=int)
    '''
print('finished')



snapnr: 74
01:29:52.757571
[ 15.85394859  15.28774834  16.39402962]
Found 754347 stars.
Rotated pos.
Rotated vel.
9785
01:30:09.383267
0
01:30:09.455216
1000
01:30:09.518093
2000
01:30:09.584885
3000
01:30:09.657316
4000
01:30:09.723668
5000
01:30:09.790180
6000
01:30:09.870810
7000
01:30:09.950547
8000
01:30:10.022996
9000
numbers of GCs wo actions: 0
0
snapnr: 75
01:30:10.088399
[ 16.16160011  15.58262157  16.72860718]
Found 788548 stars.
Rotated pos.
Rotated vel.
9785
01:30:25.894296
0
01:30:25.960179
1000
01:30:26.029092
2000
01:30:26.097980
3000
01:30:26.171402
4000
01:30:26.237069
5000
01:30:26.306445
6000
01:30:26.371789
7000
01:30:26.453066
8000
01:30:26.532379
9000
numbers of GCs wo actions: 0
0
snapnr: 76
01:30:26.597682
[ 16.47659492  15.88422775  17.06827354]
Found 817842 stars.
Rotated pos.
Rotated vel.
9785
01:30:42.644420
0
01:30:42.713283
1000
01:30:42.793807
2000
01:30:42.860817
3000
01:30:42.941818
4000
01:30:43.001743
5000
01:30:43.082018
6000
01:30:43.162571
7000
01